In [1]:
# Imports
import pandas as pd
import os
import datetime

In [2]:
# Get list of file name with path to csv
def get_csv_file_path():
  csv_file_list = sorted(os.listdir(path="./Raw-Backups/"))
  location = "./Raw-Backups/{}"
  return [
    location.format(file_name)
    for file_name in csv_file_list
    if file_name.endswith(".csv")
  ]

In [3]:
# Recieve filename and return datetime of file creation
def parse_datetime_from_filename(filename):
  try:
    # Extract datetime string from filename: 'kenmei-export-2022-08-27T17_19_00Z.csv'
    dt_str = filename.split('kenmei-export-')[-1].replace('.csv', '').replace('_', ':').replace('T', ' ')
    return datetime.datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%SZ")
  except Exception as e:
    raise ValueError(f"Could not parse datetime from filename '{filename}': {e}")

In [4]:
# Index series_url column to use as PK
def enforce_primary_key(df, key_column=""):
  if df[key_column].isnull().any():
    raise ValueError(f"Primary key column '{key_column}' contains null values.")

  if not df[key_column].is_unique:
    raise ValueError(f"Primary key constraint violated: duplicates found in '{key_column}'.")

  return df.set_index(key_column, drop=False)

In [5]:
# Update merge backup file with data from new file and override rows where PK already exists
def upsert_on_primary_key(master_df, new_df, pk="series_url"):
    # Set PK as index and validate uniqueness
    master_df = enforce_primary_key(master_df, key_column=pk)
    new_df = enforce_primary_key(new_df, key_column=pk)

    # Drop overlapping rows from master_df
    overlapping_keys = master_df.index.intersection(new_df.index)
    if not overlapping_keys.empty:
        master_df = master_df.drop(index=overlapping_keys)

    # Combine both — new rows overwrite old ones
    combined_df = pd.concat([master_df, new_df])

    # Optional: sort by key
    return combined_df.sort_index()

In [6]:
# Read merged backup file, if merged backup file doesn't exist create it
def load_history(filepath):
  DATA_COLUMNS = [
    "title", "status", "last_volume_read", "last_chapter_read",
    "last_chapter_title_read", "last_read_at", "tracked_site", "series_url",
    "migratable", "source_to_be_removed_at", "notes", "tags", "score"
  ]

  HISTORY_COLUMNS = ["filename", "file_datetime", "imported_at"]

  if os.path.exists(filepath):
    sheet_data = pd.read_excel(filepath, sheet_name=None)

    # Pad Data if any columns are missing
    if "Data" in sheet_data:
      df = sheet_data["Data"]
      for col in DATA_COLUMNS:
        if col not in df.columns:
          df[col] = pd.NA
      sheet_data["Data"] = df[DATA_COLUMNS]  # reorder columns
    else:
      sheet_data["Data"] = pd.DataFrame(columns=DATA_COLUMNS)

    # Ensure ImportHistory at least has default structure
    if "ImportHistory" not in sheet_data:
      sheet_data["ImportHistory"] = pd.DataFrame(columns=HISTORY_COLUMNS)

    return sheet_data

  else:
    return {
      "Data": pd.DataFrame(columns=DATA_COLUMNS),
      "ImportHistory": pd.DataFrame(columns=HISTORY_COLUMNS)
    }

In [7]:
# Merge source_to_be_removed_at and source_removed_at columns and delete source_removed_at column
def merge_source_columns(df):
  # If both columns exist, merge them
  if "source_removed_at" in df.columns and "source_to_be_removed_at" in df.columns:
    # Fill NaNs in source_to_be_removed_at with values from source_removed_at
    df["source_to_be_removed_at"] = df["source_to_be_removed_at"].combine_first(df["source_removed_at"])
    # Drop the old column
    df.drop(columns=["source_removed_at"], inplace=True)

  return df.sort_values("title", inplace=True)

In [8]:
# Save merged backup file file
def save_master(filepath, master_df, history_df):
  with pd.ExcelWriter(filepath, engine='openpyxl', mode='w') as writer:
    master_df.to_excel(writer, index=False, sheet_name="Data")
    history_df.to_excel(writer, index=False, sheet_name="ImportHistory")

In [9]:
# Recursively update merged backup file
def recursive_merge(files, idx, master_df, history_df, master_path):
  if idx >= len(files):
    print("✅ All files processed.")
    master_df = merge_source_columns(master_df)
    save_master(master_path, master_df, history_df)
    return

  current_file = files[idx]
  file_dt = parse_datetime_from_filename(current_file)
  filename = os.path.basename(current_file)

  # Skip if already imported
  if filename in history_df["filename"].values:
    print(f"⚠️ {filename} already imported. Skipping.")
    return recursive_merge(files, idx + 1, master_df, history_df, master_path)

  print(f"📂 Importing {filename}")
  new_df = pd.read_csv(current_file)

  # Ensure both DataFrames have same columns
  all_columns = set(master_df.columns).union(set(new_df.columns))

  for col in all_columns:
    if col not in master_df.columns:
      master_df[col] = None
    if col not in new_df.columns:
      new_df[col] = None

  # Replace manual loop with upsert
  master_df = upsert_on_primary_key(master_df, new_df, pk="series_url")

  # Record import in history
  history_df = pd.concat([history_df, pd.DataFrame([{
    "filename": filename,
    "file_datetime": file_dt,
    "imported_at": datetime.datetime.now()
  }])], ignore_index=True)

  return recursive_merge(files, idx + 1, master_df, history_df, master_path)


In [ ]:
# Define merged backup file
backup_filename = "MasterBackup"
backup_file = f"{backup_filename}.xlsx"

# Step 1: Get all CSVs and sort them by datetime extracted from filename
all_csvs = get_csv_file_path()

# Step 2: Load existing master and history
loaded = load_history(backup_file)
master_df = loaded["Data"]
history_df = loaded["ImportHistory"]

# Step 3: Start recursive import
recursive_merge(all_csvs, 0, master_df, history_df, backup_file)